# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [3]:
player_df = pd.read_csv("fifa19.csv")

In [4]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']

In [5]:
player_df = player_df[numcols+catcols]
print(player_df)

       Overall  Crossing  Finishing  ShortPassing  Dribbling  LongPassing  \
0           94      84.0       95.0          90.0       97.0         87.0   
1           94      84.0       94.0          81.0       88.0         77.0   
2           92      79.0       87.0          84.0       96.0         78.0   
3           91      17.0       13.0          50.0       18.0         51.0   
4           91      93.0       82.0          92.0       86.0         91.0   
...        ...       ...        ...           ...        ...          ...   
18202       47      34.0       38.0          49.0       42.0         45.0   
18203       47      23.0       52.0          43.0       39.0         25.0   
18204       47      25.0       40.0          38.0       45.0         28.0   
18205       47      44.0       50.0          42.0       51.0         32.0   
18206       46      41.0       34.0          48.0       43.0         44.0   

       BallControl  Acceleration  SprintSpeed  Agility  ...  ShotPower  \
0

In [6]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [7]:
traindf = pd.DataFrame(traindf,columns=features)
print(traindf)

       Overall  Crossing  Finishing  ShortPassing  Dribbling  LongPassing  \
0           94      84.0       95.0          90.0       97.0         87.0   
1           94      84.0       94.0          81.0       88.0         77.0   
2           92      79.0       87.0          84.0       96.0         78.0   
3           91      17.0       13.0          50.0       18.0         51.0   
4           91      93.0       82.0          92.0       86.0         91.0   
...        ...       ...        ...           ...        ...          ...   
18202       47      34.0       38.0          49.0       42.0         45.0   
18203       47      23.0       52.0          43.0       39.0         25.0   
18204       47      25.0       40.0          38.0       45.0         28.0   
18205       47      44.0       50.0          42.0       51.0         32.0   
18206       46      41.0       34.0          48.0       43.0         44.0   

       BallControl  Acceleration  SprintSpeed  Agility  ...  \
0           

In [8]:
y = traindf['Overall']>=87
X = traindf.copy()
del X['Overall']

In [9]:
print(X.head())

   Crossing  Finishing  ShortPassing  Dribbling  LongPassing  BallControl  \
0      84.0       95.0          90.0       97.0         87.0         96.0   
1      84.0       94.0          81.0       88.0         77.0         94.0   
2      79.0       87.0          84.0       96.0         78.0         95.0   
3      17.0       13.0          50.0       18.0         51.0         42.0   
4      93.0       82.0          92.0       86.0         91.0         91.0   

   Acceleration  SprintSpeed  Agility  Stamina  ...  Nationality_Uganda  \
0          91.0         86.0     91.0     72.0  ...                   0   
1          89.0         91.0     87.0     88.0  ...                   0   
2          94.0         90.0     96.0     81.0  ...                   0   
3          57.0         58.0     60.0     43.0  ...                   0   
4          78.0         76.0     79.0     90.0  ...                   0   

   Nationality_Ukraine  Nationality_United Arab Emirates  \
0                    0    

In [10]:
print(len(X.columns))

223


### Set some fixed set of features

In [13]:
feature_names = list(X.columns)
# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation

### Pearson Correlation function

In [14]:
def cor_selector(X, y,num_feats):
    # Your code goes here (Multiple lines)
    #list your coefficients
    cor_list = []
    for i in feature_names:
        cor = np.corrcoef(X[i], y)[0,1]
        cor_list.append(cor)
    #Imputation
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    #features from best feature
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    #get bool values( Match True and Flase values)
    cor_support = [True if i in cor_feature else False for i in feature_names]
    
    # Your code ends here
    return cor_support, cor_feature

In [15]:
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [16]:
print(cor_feature)

['Nationality_Costa Rica', 'Position_LAM', 'Nationality_Uruguay', 'Acceleration', 'SprintSpeed', 'Strength', 'Nationality_Gabon', 'Nationality_Slovenia', 'Stamina', 'Weak Foot', 'Agility', 'Crossing', 'Nationality_Belgium', 'Dribbling', 'ShotPower', 'LongShots', 'Finishing', 'BallControl', 'FKAccuracy', 'LongPassing', 'Volleys', 'ShortPassing', 'Position_RF', 'Position_LF', 'Body Type_PLAYER_BODY_TYPE_25', 'Body Type_Courtois', 'Body Type_Neymar', 'Body Type_Messi', 'Body Type_C. Ronaldo', 'Reactions']


## Filter Feature Selection - Chi-Sqaure

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

### Chi-Squared Selector function

In [18]:
min_max = MinMaxScaler().fit_transform(X)
print(min_max)

[[0.89772727 1.         0.96511628 ... 0.         0.         0.        ]
 [0.89772727 0.98924731 0.86046512 ... 0.         0.         0.        ]
 [0.84090909 0.91397849 0.89534884 ... 0.         0.         0.        ]
 ...
 [0.22727273 0.40860215 0.36046512 ... 0.         0.         0.        ]
 [0.44318182 0.51612903 0.40697674 ... 0.         0.         0.        ]
 [0.40909091 0.34408602 0.47674419 ... 0.         0.         0.        ]]


In [19]:
def chi_squared_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    #Standardize the fifa data with min max scale
    chi = SelectKBest(score_func=chi2, k=num_feats).fit(min_max,y)
    #get Bool values
    chi_support = chi.get_support()
    # get columns
    chi_feature = X.iloc[:,chi_support].columns.tolist()
    # Your code ends here
    return chi_support, chi_feature

In [20]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square 

In [21]:
print(chi_feature)

['Finishing', 'ShortPassing', 'LongPassing', 'BallControl', 'Volleys', 'FKAccuracy', 'Reactions', 'LongShots', 'Position_CM', 'Position_LAM', 'Position_LF', 'Position_LW', 'Position_RB', 'Position_RF', 'Body Type_C. Ronaldo', 'Body Type_Courtois', 'Body Type_Messi', 'Body Type_Neymar', 'Body Type_PLAYER_BODY_TYPE_25', 'Nationality_Belgium', 'Nationality_Costa Rica', 'Nationality_Croatia', 'Nationality_Egypt', 'Nationality_England', 'Nationality_France', 'Nationality_Gabon', 'Nationality_Slovakia', 'Nationality_Slovenia', 'Nationality_Spain', 'Nationality_Uruguay']


## Wrapper Feature Selection - Recursive Feature Elimination

In [22]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [23]:
def rfe_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    #NB use step of 10 
    rfe = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=1).fit(min_max,y)
    rfe_support = rfe.get_support()
    rfe_feature = X.iloc[:,rfe_support].columns.tolist()
    # Your code ends here
    return rfe_support, rfe_feature

In [24]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.
30 selected features


### List the selected features from RFE

In [25]:
print(rfe_feature)

['Finishing', 'ShortPassing', 'LongPassing', 'BallControl', 'SprintSpeed', 'Agility', 'Volleys', 'FKAccuracy', 'Reactions', 'Strength', 'Weak Foot', 'Position_CAM', 'Position_CM', 'Position_GK', 'Position_LCB', 'Position_LM', 'Position_RB', 'Position_RCB', 'Position_RF', 'Position_RM', 'Position_RW', 'Body Type_Courtois', 'Body Type_PLAYER_BODY_TYPE_25', 'Nationality_Belgium', 'Nationality_Costa Rica', 'Nationality_Croatia', 'Nationality_Gabon', 'Nationality_Netherlands', 'Nationality_Slovenia', 'Nationality_Uruguay']


## Embedded Selection - Lasso: SelectFromModel

In [26]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [27]:
def embedded_log_reg_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    embedded = SelectFromModel(LogisticRegression(penalty='l2',solver='lbfgs'), max_features=num_feats).fit(min_max,y)
    embedded_lr_support = embedded.get_support()
    embedded_lr_feature = X.iloc[:,embedded_lr_support].columns.tolist()
    # Your code ends here
    return embedded_lr_support, embedded_lr_feature

In [28]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')

30 selected features


In [29]:
print(embedded_lr_feature)

['Finishing', 'ShortPassing', 'LongPassing', 'BallControl', 'SprintSpeed', 'Agility', 'Volleys', 'Reactions', 'Strength', 'Weak Foot', 'Position_CAM', 'Position_CM', 'Position_GK', 'Position_LCB', 'Position_LM', 'Position_RB', 'Position_RCB', 'Position_RM', 'Position_RW', 'Body Type_Courtois', 'Body Type_Lean', 'Body Type_Stocky', 'Nationality_Belgium', 'Nationality_Costa Rica', 'Nationality_Croatia', 'Nationality_France', 'Nationality_Gabon', 'Nationality_Netherlands', 'Nationality_Slovenia', 'Nationality_Uruguay']


## Tree based(Random Forest): SelectFromModel

In [30]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [31]:
def embedded_rf_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    rf = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats).fit(min_max,y)
    embedded_rf_support = rf.get_support()
    embedded_rf_feature = X.iloc[:,embedded_rf_support].columns.tolist()
    # Your code ends here
    return embedded_rf_support, embedded_rf_feature

In [32]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedded_rf_feature)), 'selected features')

23 selected features


In [33]:
print(embedded_rf_feature)

['Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina', 'Volleys', 'FKAccuracy', 'Reactions', 'Balance', 'ShotPower', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Weak Foot', 'Body Type_Courtois', 'Nationality_Belgium', 'Nationality_Slovenia']


## Tree based(Light GBM): SelectFromModel

In [34]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [35]:
def embedded_lgbm_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    lgbm = SelectFromModel(LGBMClassifier(n_estimators=500), max_features=num_feats).fit(min_max,y)
    embedded_lgbm_support = lgbm.get_support()
    embedded_lgbm_feature = X.iloc[:,embedded_lgbm_support].columns.tolist()
    # Your code ends here
    return embedded_lgbm_support, embedded_lgbm_feature

In [36]:
embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
print(str(len(embedded_lgbm_feature)), 'selected features')

22 selected features


In [37]:
print(embedded_lgbm_feature)

['Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina', 'Volleys', 'FKAccuracy', 'Reactions', 'Balance', 'ShotPower', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Position_LCB', 'Body Type_Lean', 'Nationality_Italy']


## Putting all of it together: AutoFeatureSelector Tool

In [39]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
print(feature_selection_df.head(num_feats))

                          Feature  Pearson  Chi-2    RFE  Logistics  \
1                         Volleys     True   True   True       True   
2                    ShortPassing     True   True   True       True   
3                       Reactions     True   True   True       True   
4                     LongPassing     True   True   True       True   
5                       Finishing     True   True   True       True   
6                     BallControl     True   True   True       True   
7                        Strength     True  False   True       True   
8                     SprintSpeed     True  False   True       True   
9            Nationality_Slovenia     True   True   True       True   
10            Nationality_Belgium     True   True   True       True   
11                     FKAccuracy     True   True   True      False   
12             Body Type_Courtois     True   True   True       True   
13                        Agility     True  False   True       True   
14    

## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [40]:
def preprocess_dataset(dataset_path):
    # Your code starts here (Multiple lines)
    player_df = pd.read_csv("fifa19.csv")
    numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
    catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']
    player_df = player_df[numcols+catcols]
    traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
    features = traindf.columns
    traindf = traindf.dropna()
    traindf = pd.DataFrame(traindf,columns=features)
    y = traindf['Overall']>=87
    X = traindf.copy()
    del X['Overall']
    # Your code ends here
    return X, y, num_feats

In [44]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)
    
    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)
    
    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
    
    
# Combine all the above feature list and count the maximum set of features that got selected by all methods
#### Your Code starts here (Multiple lines)
    feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
# count the selected times for each feature
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    bf = feature_selection_df.head(num_feats)
    best_features = bf.iloc[:10,0]
    
    
    #### Your Code ends here
    return best_features

In [45]:
best_features = autoFeatureSelector(dataset_path="fifa19.csv", methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm'])
print(best_features)

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.
1                    Volleys
2               ShortPassing
3                  Reactions
4                LongPassing
5                  Finishing
6                BallControl
7                   Strength
8                SprintSpeed
9       Nationality_Slovenia
10    

### Last, Can you turn this notebook into a python script, run it and submit the python (.py) file that takes dataset and list of methods as inputs and outputs the best features